## Fetch hourly weather for trips
This notebook loads cleaned trip data, rounds start times to the nearest hour, queries Open-Meteo for temperature, rain, and weather code, and merges the results back to the trips.

In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed

import sys
import os

# Add the parent directory to sys.path so 'src' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.db import open_connection
import pandas as pd

conn = open_connection()

trips_df = pd.read_sql_query('SELECT * FROM public.trips LIMIT 40000;', conn)
trips_df

/tmp/ipykernel_161922/256836955.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trips_df = pd.read_sql_query('SELECT * FROM public.trips LIMIT 40000;', conn)


,id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,bike_type
0,396993,22,2024-04-24 17:39:00,2024-04-24 18:01:00,3005,34.048500,-118.258537,4443,34.022091,-118.279793,26417,electric
1,396994,1,2024-04-24 17:39:00,2024-04-24 17:40:00,3048,34.041691,-118.235352,3048,34.041691,-118.235352,13856,standard
2,396995,6,2024-04-24 17:39:00,2024-04-24 17:45:00,3030,34.051941,-118.243530,3014,34.056610,-118.237213,12882,standard
3,396996,1,2024-04-24 17:40:00,2024-04-24 17:40:00,4632,33.986671,-118.440948,4632,33.986671,-118.440948,20175,standard
4,396997,55,2024-04-24 17:40:00,2024-04-24 18:35:00,4632,33.986671,-118.440948,4215,34.014309,-118.491341,15085,standard
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,436988,48,2024-05-18 16:12:00,2024-05-18 17:00:00,4514,34.046211,-118.451172,4214,33.995560,-118.481552,5789,standard
39996,436989,26,2024-05-18 16:13:00,2024-05-18 16:39:00,4212,33.988129,-118.471741,4215,34.014309,-118.491341,23969,electric
39997,436990,21,2024-05-18 16:13:00,2024-05-18 16:34:00,4555,34.005871,-118.429161,4211,33.984928,-118.469963,30392,electric
39998,436991,27,2024-05-18 16:13:00,2024-05-18 16:40:00,4212,33.988129,-118.471741,4215,34.014309,-118.491341,24389,electric


In [ ]:
import pandas as pd
import numpy as np
import requests_cache

# 1) Trips einlesen und Merge-Keys bauen
df = trips_df.copy()
df['date_only']         = df['start_time'].dt.strftime('%Y-%m-%d')
df['time_only_rounded'] = df['start_time'].dt.round('h').dt.strftime('%H:%M:%S')

# 2) Unique (lat, lon) extrahieren
stations = (
    df[['start_lat','start_lon']]
      .drop_duplicates()
      .reset_index(drop=True)
)

# 3) In Batches aufteilen (z.B. 50 Stationen pro Request)
chunk_size = 50
batches = [
    stations.iloc[i:i+chunk_size].to_dict('records')
    for i in range(0, len(stations), chunk_size)
]

# 4) Session mit Cache (1h)
session = requests_cache.CachedSession('.cache', expire_after=3600)

# 5) Batch-Fetcher
def fetch_batch(batch):
    # Koordinaten als Komma-Strings
    lat_str = ",".join(str(p['start_lat']) for p in batch)
    lon_str = ",".join(str(p['start_lon']) for p in batch)

    params = {
        "latitude":   lat_str,
        "longitude":  lon_str,
        "start_date": df['date_only'].min(),
        "end_date":   df['date_only'].max(),
        "hourly":     "temperature_2m,rain,weathercode",
        "timezone":   "auto",   # → liefert lokale Iso-Strings
    }

    # JSON-Request
    url  = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    r    = session.get(url, params=params)
    r.raise_for_status()
    js   = r.json()

    # Falls das API-Resultat eine Liste ist, nimm diese,
    # sonst wickle das Einzel-Objekt in eine Liste
    forecasts = js if isinstance(js, list) else [js]

    # Die Zeit‐Achse ist für alle Punkte gleich:
    times = pd.to_datetime(forecasts[0]["hourly"]["time"])
    n     = len(times)

    records = []
    for idx, loc in enumerate(batch):
        data_hourly = forecasts[idx]["hourly"]

        temp_arr = np.array(data_hourly["temperature_2m"])
        rain_arr = np.array(data_hourly["rain"])
        code_arr = np.array(data_hourly["weathercode"])

        # Datum/Uhrzeit-Strings in lokaler Zeit
        date_strs = times.strftime('%Y-%m-%d')
        time_strs = times.strftime('%H:%M:%S')

        # Baue Mini-DataFrame für diese Station
        df_loc = pd.DataFrame({
            "start_lat":         [loc['start_lat']] * n,
            "start_lon":         [loc['start_lon']] * n,
            "date_only":         date_strs,
            "time_only_rounded": time_strs,
            "temperature_2m":    temp_arr,
            "rain":              rain_arr,
            "weather_code":      code_arr,
        })
        records.append(df_loc)

    return records

# 6) Parallel alle Batches abarbeiten
all_dfs = []
with ThreadPoolExecutor(max_workers=5) as exe:
    futures = [exe.submit(fetch_batch, b) for b in batches]
    for fut in as_completed(futures):
        all_dfs.extend(fut.result())

# 7) Zu einem DataFrame zusammenführen
weather_full = pd.concat(all_dfs, ignore_index=True)

# 8) Rückmerge ins Original-Frame
result = df.merge(
    weather_full,
    on=['start_lat','start_lon','date_only','time_only_rounded'],
    how='left'
)

# 9) Kontrolle
print(result.head())


HTTPError: 429 Client Error: Too Many Requests for url: https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=34.092602%2C34.051941%2C33.976189%2C34.04554%2C34.00351%2C34.04417%2C34.046131%2C34.03017%2C33.986671%2C33.928459%2C33.989079%2C33.984341%2C34.10363%2C34.023392%2C33.99556%2C34.046211%2C34.039982%2C33.988129%2C34.0532%2C33.998341%2C33.984928%2C34.048401%2C34.06163%2C33.996239%2C34.014309%2C34.035351%2C33.95879%2C34.049301%2C34.02404%2C34.094349%2C34.063179%2C34.01376%2C34.069752%2C34.056278%2C34.01252%2C34.04298%2C34.04744%2C33.99868%2C34.02697%2C33.991161%2C34.05088%2C34.02586%2C34.04998%2C34.027618%2C34.06097%2C34.04652%2C33.997238%2C34.057709%2C34.043522%2C34.082649&longitude=-118.28093%2C-118.24353%2C-118.418419%2C-118.256668%2C-118.43438%2C-118.261169%2C-118.257591%2C-118.280952%2C-118.440948%2C-118.435204%2C-118.462257%2C-118.47155%2C-118.291313%2C-118.399223%2C-118.481552%2C-118.451172%2C-118.266403%2C-118.471741%2C-118.250954%2C-118.461014%2C-118.469963%2C

: 

In [ ]:
result[result['rain'] > 0]

,id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,bike_type,date_only,time_only_rounded,temperature_2m,rain,weather_code


In [ ]:

# 1) Calendar week (ISO standard, 1–53)
result['calendar_week'] = result['start_time'].dt.isocalendar().week

# 2) Weekday name and number (Monday=1 … Sunday=7)
result['weekday_name'] = result['start_time'].dt.day_name()        # e.g. "Monday"
result['weekday_num']  = result['start_time'].dt.weekday + 1        # Monday=1 … Sunday=7

# 3) Season (meteorological: Winter, Spring, Summer, Fall)
seasons = {0: 'Winter', 1: 'Spring', 2: 'Summer', 3: 'Fall'}
# (month % 12)//3 → 0 for Dec–Feb, 1 for Mar–May, 2 for Jun–Aug, 3 for Sep–Nov
result['season'] = (result['start_time'].dt.month % 12 // 3).map(seasons)

# 4) Time of day category
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

result['time_of_day'] = result['start_time'].dt.hour.map(time_of_day)

# 5) Weekend indicator
def is_weekend(weekday_num):
    return weekday_num >= 6  # Saturday=6, Sunday=7

result['is_weekend'] = result['weekday_num'].map(is_weekend)

# Inspect the new columns



In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 30 non-null     int64         
 1   duration           30 non-null     int64         
 2   start_time         30 non-null     datetime64[ns]
 3   end_time           30 non-null     datetime64[ns]
 4   start_station      30 non-null     int64         
 5   start_lat          30 non-null     float64       
 6   start_lon          30 non-null     float64       
 7   end_station        30 non-null     int64         
 8   end_lat            30 non-null     float64       
 9   end_lon            30 non-null     float64       
 10  bike_id            30 non-null     int64         
 11  bike_type          30 non-null     object        
 12  date_only          30 non-null     object        
 13  time_only_rounded  30 non-null     object        
 14  temperature_

In [ ]:
result[result["is_weekend"]==1]

,id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,...,time_only_rounded,temperature_2m,rain,weather_code,calendar_week,weekday_name,weekday_num,season,time_of_day,is_weekend


In [ ]:
%pip install scikit-learn

import pandas as pd
from sklearn.cluster import KMeans

# ------------- 1. Kopie & Grund-Features ------------------
df = result.copy()                                # ← dein Trip-DataFrame
df['slot_ts'] = df['start_time'].dt.floor('h')     # stündliche Slots
df['hour_of_day'] = df['slot_ts'].dt.hour
df['weekday_num'] = df['slot_ts'].dt.weekday
df['is_weekend']  = df['weekday_num'].isin([5, 6])

# Temperatur in Klassen einteilen
def temp_class(t):
    if t < 10:   return 'cold'
    if t < 20:   return 'mid'
    if t < 28:   return 'warm'
    return 'heiss'

df['temp_class'] = df['temperature_2m'].apply(temp_class)
df['is_raining'] = df['rain'] >= 0.1

# ------------- 2. Stations-Stammdaten & Cluster ------------
stations = (
    df[['start_station', 'start_lat', 'start_lon']]
      .drop_duplicates()
      .rename(columns={'start_station': 'station_id',
                       'start_lat': 'lat',
                       'start_lon': 'lon'})
)

# einfache räumliche Cluster-ID (80 Cluster als Beispiel)
kmeans = KMeans(n_clusters=80, random_state=0)
stations['cluster_id'] = kmeans.fit_predict(stations[['lat', 'lon']])

# ------------- 3. Bikes taken / returned pro Stunde --------
taken = (df.groupby(['slot_ts', 'start_station'])
           .size().reset_index(name='bikes_taken'))

returned = (df.groupby(['slot_ts', 'end_station'])
             .size().reset_index(name='bikes_returned'))

agg = (pd.merge(taken, returned,
                left_on=['slot_ts', 'start_station'],
                right_on=['slot_ts', 'end_station'],
                how='outer')
         .rename(columns={'start_station': 'station_id'})
       )

# station_id von rechten Merge übernehmen, fehlende Counts = 0
agg['station_id'] = agg['station_id'].fillna(agg['end_station']).astype(int)
agg = agg.drop(columns='end_station')
agg[['bikes_taken', 'bikes_returned']] = agg[['bikes_taken',
                                             'bikes_returned']].fillna(0).astype(int)

# ------------- 4. Wetter stündlich mitteln -----------------
weather_hourly = (df.groupby('slot_ts')
                    .agg(temperature_2m=('temperature_2m', 'mean'),
                         temp_class=('temp_class',
                                     lambda x: x.mode().iloc[0] if not x.mode().empty else None),
                         rain_mm=('rain', 'mean'),
                         is_raining=('is_raining', 'max'),
                         weather_code=('weather_code',
                                       lambda x: x.mode().iloc[0] if not x.mode().empty else None))
                    .reset_index())

# ------------- 5. Alles zusammenführen --------------------
agg = (agg
       .merge(stations,       on='station_id', how='left')
       .merge(weather_hourly, on='slot_ts',    how='left'))

# Reihenfolge & Saisonzuteilung
season_map = {12:'Winter', 1:'Winter', 2:'Winter',
              3:'Spring', 4:'Spring', 5:'Spring',
              6:'Summer', 7:'Summer', 8:'Summer',
              9:'Fall', 10:'Fall', 11:'Fall'}
agg['season'] = agg['slot_ts'].dt.month.map(season_map)

# ------------- 6. SQL-Typ-Mapping printen ------------------
sql_map = {
    'int64':        'INTEGER',
    'int32':        'INTEGER',
    'float64':      'REAL',
    'bool':         'BOOLEAN',
    'datetime64[ns]':'TIMESTAMP',
    'object':       'TEXT',
    'category':     'VARCHAR'
}

print("\nSpalten  |  Pandas-dtype  →  SQL-Typ")
print("-" * 45)
for col, dt in agg.dtypes.items():
    print(f"{col:<15} {str(dt):<15} →  {sql_map.get(str(dt), 'TEXT')}")

print("\nBeispiel-Rows:")
print(agg.head())


Note: you may need to restart the kernel to use updated packages.


ValueError: n_samples=26 should be >= n_clusters=80.

In [ ]:
agg

,slot_ts,station_id,bikes_taken,bikes_returned,lat,lon,cluster_id,temperature_2m,temp_class,rain_mm,is_raining,weather_code,season
0,2025-01-01 00:00:00,3026,0,1,34.063179,-118.245880,17,10.200000,mid,0.0,False,3,Winter
1,2025-01-01 00:00:00,3028,1,0,34.058319,-118.246094,17,10.200000,mid,0.0,False,3,Winter
2,2025-01-01 00:00:00,3030,1,0,34.051941,-118.243530,53,10.200000,mid,0.0,False,3,Winter
3,2025-01-01 00:00:00,3042,1,0,34.049301,-118.238808,8,10.200000,mid,0.0,False,3,Winter
4,2025-01-01 00:00:00,3047,1,0,34.039982,-118.266403,52,10.200000,mid,0.0,False,3,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92670,2025-03-31 23:00:00,4627,0,1,34.101849,-118.325142,55,13.041176,mid,0.0,False,3,Spring
92671,2025-03-31 23:00:00,4643,2,0,34.072620,-118.449440,60,13.041176,mid,0.0,False,3,Spring
92672,2025-03-31 23:00:00,4664,0,1,34.055161,-118.308990,35,13.041176,mid,0.0,False,3,Spring
92673,2025-03-31 23:00:00,4672,0,1,34.045021,-118.253189,14,13.041176,mid,0.0,False,3,Spring
